In [ ]:
# cd to your workstation, if necessary
%cd /home/aistudio/work/

In [2]:
# 导入所需的第三方库
import numpy as np
import time
import os
import paddle
import paddle.nn.functional as F

In [ ]:
!pip install --upgrade paddlenlp==2.1

In [4]:
# 导入paddlenlp所需的相关包
import paddlenlp as ppnlp
from paddlenlp.data import Tuple, Pad

In [ ]:
# 使用roberta-wwm-ext-large模型
# MODEL_NAME = "roberta-wwm-ext-large"
# 从保存的参数中读取
MODEL_NAME = 'English_0.82_finished'
# 只需指定想要使用的模型名称和文本分类的类别数即可完成Fine-tune网络定义，通过在预训练模型后拼接上一个全连接网络（Full Connected）进行分类
model = ppnlp.transformers.ErnieForSequenceClassification.from_pretrained(MODEL_NAME, num_classes=8) # 此次分类任务为8分类任务，故num_classes设置为8
# 定义模型对应的tokenizer，tokenizer可以把原始输入文本转化成模型model可接受的输入数据格式。需注意tokenizer类要与选择的模型相对应，具体可以查看PaddleNLP相关文档
tokenizer = ppnlp.transformers.ErnieTokenizer.from_pretrained(MODEL_NAME)

In [6]:
# 动转静，通过`input_spec`给出模型所需输入数据的描述，shape中的None代表可变的大小，类似上面静态图模式中的`paddle.static.data`
model.eval()
model = paddle.jit.to_static(
        model,
        input_spec=[
            paddle.static.InputSpec(shape=[None, None],
                                    dtype="int64"),  # input_ids
            paddle.static.InputSpec(shape=[None, None],
                                    dtype="int64")  # segment_ids
        ])

In [7]:
# 保存动转静后的模型，得到 infer_model/model.pdmodel 和 infer_model/model.pdiparams 文件
infer_model_save_path = os.path.join(MODEL_NAME, 'infer_model/model')
paddle.jit.save(model, infer_model_save_path)